In [1]:
from tensorflow import keras
import numpy as np
import nltk
from copy import deepcopy
from scipy.spatial import distance

In [2]:
text = 'Once upon a time, there lived two monarchs. The king sat on a throne. Also, the queen sat on a throne. Together, they both were seated on thrones.'
NGRAM_LIMIT = 10000
tokens = nltk.tokenize.word_tokenize(text)
tokens_lc = [x.lower() for x in tokens]
INDEXES = {x: i for i, x in enumerate(sorted(set(tokens_lc)))}
REV_INDEXES = {x: i for i, x in INDEXES.items()}
bigrams = list(nltk.bigrams(tokens_lc))
bigrams = bigrams[:min(len(bigrams), NGRAM_LIMIT)]

In [3]:
x_text = [w1 for w1, w2 in bigrams]
y_text = [w2 for w1, w2 in bigrams]

x_num = [INDEXES[x] for x in x_text]
y_num = [INDEXES[y] for y in y_text]

X = keras.utils.to_categorical(x_num, num_classes=len(INDEXES))
y = keras.utils.to_categorical(y_num, num_classes=len(INDEXES))

In [4]:
m = keras.models.Sequential()
m.add(keras.layers.Dense(64, activation='relu', input_dim=len(INDEXES)))
m.add(keras.layers.Dense(32, activation='relu'))
m.add(keras.layers.Dense(len(INDEXES), activation='softmax'))
m.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')
m.summary()
m.fit(X, y, epochs=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1536      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 23)                759       
                                                                 
Total params: 4,375
Trainable params: 4,375
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2/2 [==============================] - 0s 4ms/step - loss: 3.1402 - acc: 0.0294
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 3.1250 - acc: 0.0588
Epoch 3/100
2/2 [==============================] - 0s 4ms/step - loss: 3.1132 - acc: 0.0588
Epoch 4/100
2/2 [=========================

2/2 [==============================] - 0s 4ms/step - loss: 2.0682 - acc: 0.5882
Epoch 81/100
2/2 [==============================] - 0s 4ms/step - loss: 2.0466 - acc: 0.5882
Epoch 82/100
2/2 [==============================] - 0s 4ms/step - loss: 2.0259 - acc: 0.5882
Epoch 83/100
2/2 [==============================] - 0s 4ms/step - loss: 2.0056 - acc: 0.5882
Epoch 84/100
2/2 [==============================] - 0s 4ms/step - loss: 1.9871 - acc: 0.6176
Epoch 85/100
2/2 [==============================] - 0s 8ms/step - loss: 1.9685 - acc: 0.6176
Epoch 86/100
2/2 [==============================] - 0s 3ms/step - loss: 1.9499 - acc: 0.6176
Epoch 87/100
2/2 [==============================] - 0s 0s/step - loss: 1.9306 - acc: 0.6176
Epoch 88/100
2/2 [==============================] - 0s 4ms/step - loss: 1.9127 - acc: 0.6176
Epoch 89/100
2/2 [==============================] - 0s 4ms/step - loss: 1.8961 - acc: 0.6176
Epoch 90/100
2/2 [==============================] - 0s 4ms/step - loss: 1.8790 - acc

In [5]:
newmod = keras.models.clone_model(m)
newmod.pop() # Remove last layer

In [6]:
def get_vector(word):
    i = INDEXES[word]
    onehot = keras.utils.to_categorical([i], num_classes=len(INDEXES))
    return newmod.predict(onehot)

In [7]:
print(get_vector('queen'))

[[0.05006447 0.         0.06754407 0.08911205 0.         0.13062237
  0.11237837 0.         0.13539109 0.04135512 0.09900975 0.
  0.         0.00148174 0.07647885 0.         0.03603644 0.
  0.2021379  0.         0.06547876 0.         0.         0.
  0.         0.2897411  0.1228924  0.         0.         0.1009649
  0.         0.        ]]


In [8]:
print(get_vector('king'))

[[0.         0.20444356 0.         0.0072428  0.01023859 0.
  0.02600182 0.         0.         0.0441102  0.         0.03709315
  0.         0.         0.11913185 0.         0.         0.
  0.14938039 0.         0.         0.         0.         0.
  0.12239395 0.23037995 0.09291933 0.05400846 0.         0.07376099
  0.         0.        ]]
